# testing tree APIs

Loading libraries and defining paths to sets already created/downloaded:

In [ ]:
import subprocess
import os
from collections import defaultdict, deque

import polars as pl
from tqdm.notebook import tqdm


from rolypoly.utils.bio.trees import TaxonomyTree



### DEBUG ARGS (for manually building, not entering via CLI):
threads = 6
log_file = "notebooks/Exprimental/trrna.log"
data_dir = "/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data"


Starting data preparation to : /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data


In [ ]:
rrna_df = rrna_df = pl.scan_csv(
    os.path.join(data_dir, "/contam/rrna/*.tab"),
    separator="@",
    has_header=False,
    null_values = ["N/A"],
    new_columns=["taxid", "header","name","sequence"],
    include_file_paths="type"
).collect()
mini_genebank = pl.read_parquet("/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/genbank_assembly_summary.parquet")
temp_df = rrna_df.join(
    mini_genebank.select(
        ["ncbi_taxonid", "organism_name"]
    ).unique(),  # seq_length
    left_on=["taxid"],
    right_on=["ncbi_taxonid"],
    how="left",
)
temp_df["organism_name"].value_counts(sort=True)


organism_name,count
str,u32
null,117964
"""Giardia duodenalis""",127
"""Globisporangium heterothallicu…",93
"""Babesia bovis""",65
"""Globisporangium sylvaticum""",61
…,…
"""Andreaea rupestris""",1
"""Tetraphis pellucida""",1
"""Anncaliia algerae""",1


## NCBI taxdump

In [ ]:
# *.dmp files are bcp-like dump from GenBank taxonomy database.

# General information.
# Field terminator is "\t|\t"
# Row terminator is "\t|\n"

# nodes.dmp file consists of taxonomy nodes. The description for each node includes the following
# fields:
# 	tax_id					-- node id in GenBank taxonomy database
#  	parent tax_id				-- parent node id in GenBank taxonomy database
#  	rank					-- rank of this node (domain, kingdom, ...) 
#  	embl code				-- locus-name prefix; not unique
#  	division id				-- see division.dmp file
#  	inherited div flag  (1 or 0)		-- 1 if node inherits division from parent
#  	genetic code id				-- see gencode.dmp file
#  	inherited GC  flag  (1 or 0)		-- 1 if node inherits genetic code from parent
#  	mitochondrial genetic code id		-- see gencode.dmp file
#  	inherited MGC flag  (1 or 0)		-- 1 if node inherits mitochondrial gencode from parent
#  	GenBank hidden flag (1 or 0)            -- 1 if name is suppressed in GenBank entry lineage
#  	hidden subtree root flag (1 or 0)       -- 1 if this subtree has no sequence data yet
#  	comments				-- free-text comments and citations

# Taxonomy names file (names.dmp):
# 	tax_id					-- the id of node associated with this name
# 	name_txt				-- name itself
# 	unique name				-- the unique variant of this name if name not unique
# 	name class				-- (synonym, common name, ...)


INFO     2025-12-02 19:16:59 - Downloading https://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz to                                  ]8;id=203324;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=200555;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz                                                                

INFO     2025-12-02 19:17:06 - Successfully downloaded to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz ]8;id=382276;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=849077;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\

INFO     2025-12-02 19:17:06 - Extracting /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump.tar.gz to              ]8;id=127845;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=832875;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#255\255]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump                                                                       

INFO     2025-12-02 19:17:09 - Successfully extracted to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump         ]8;id=591732;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=279401;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#288\288]8;;\

PosixPath('/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/taxdump')

In [12]:
taxonomy_dir = os.path.join(data_dir, "taxdump")
nodes_path = os.path.join(taxonomy_dir, "nodes.dmp")
names_path = os.path.join(taxonomy_dir, "names.dmp")
# can not read with polars cause of irregular separator, so using line by line read...

records = []
with open(nodes_path, "r") as f:
    nodes_lines = f.readlines()
    for line in tqdm(nodes_lines):
        line = line.rstrip("\n").rstrip("\t|")
        fields = line.split("\t|\t")
        record = {
            "tax_id": fields[0],
            "parent_tax_id": fields[1],
            "rank": fields[2],
            "embl_code": fields[3],
            "division_id": fields[4],
            "inherited_div_flag": fields[5],
            "genetic_code_id": fields[6],
            "inherited_GC_flag": fields[7],
            "mitochondrial_genetic_code_id": fields[8],
            "inherited_MGC_flag": fields[9],
            "GenBank_hidden_flag": fields[10],
            "hidden_subtree_root_flag": fields[11],
            # "comments": fields[12]
        }
        records.append(record)
nodes_df = pl.from_records(records).with_columns(
    pl.col("tax_id").cast(pl.Int64),
    pl.col("parent_tax_id").cast(pl.Int64),
    
)
nodes_df
#


  0%|          | 0/2708826 [00:00<?, ?it/s]

tax_id,parent_tax_id,rank,embl_code,division_id,inherited_div_flag,genetic_code_id,inherited_GC_flag,mitochondrial_genetic_code_id,inherited_MGC_flag,GenBank_hidden_flag,hidden_subtree_root_flag
i64,i64,str,str,str,str,str,str,str,str,str,str
1,1,"""no rank""","""""","""8""","""0""","""1""","""0""","""0""","""0""","""0""","""0"""
2,131567,"""domain""","""""","""0""","""0""","""11""","""0""","""0""","""0""","""0""","""0"""
6,335928,"""genus""","""""","""0""","""1""","""11""","""1""","""0""","""1""","""0""","""0"""
7,6,"""species""","""AC""","""0""","""1""","""11""","""1""","""0""","""1""","""1""","""0"""
9,32199,"""species""","""BA""","""0""","""1""","""11""","""1""","""0""","""1""","""1""","""0"""
…,…,…,…,…,…,…,…,…,…,…,…
3475891,47833,"""no rank""","""""","""1""","""1""","""1""","""1""","""5""","""1""","""0""","""0"""
3476007,479661,"""varietas""","""RC""","""4""","""1""","""1""","""1""","""1""","""1""","""0""","""0"""
3476068,2653920,"""genus""","""""","""4""","""1""","""1""","""1""","""1""","""1""","""0""","""0"""


In [13]:
records = []
with open(names_path, "r") as f:
    nodes_lines = f.readlines()
    for line in tqdm(nodes_lines):
        line = line.rstrip("\n").rstrip("\t|")
        fields = line.split("\t|\t")
        record = {
            "tax_id": fields[0],
            "name_txt": fields[1],
            "unique_name": fields[2],
            "name_class": fields[3],
        }
        records.append(record)
names_df = pl.from_records(records).with_columns(
    pl.col("tax_id").cast(pl.Int64),
    
)
names_df

  0%|          | 0/4495871 [00:00<?, ?it/s]

tax_id,name_txt,unique_name,name_class
i64,str,str,str
1,"""all""","""""","""synonym"""
1,"""root""","""""","""scientific name"""
2,"""Bacteria""","""Bacteria <bacteria>""","""scientific name"""
2,"""bacteria""","""bacteria <blast name>""","""blast name"""
2,"""bacteria""","""bacteria <genbank common name>""","""genbank common name"""
…,…,…,…
3476068,"""Comoliopsis Wurdack, 1984""","""""","""authority"""
3476159,"""Benthoplanidae Bezio, Paulay &…","""""","""authority"""
3476159,"""Benthoplanidae""","""""","""scientific name"""


In [19]:
test = names_df.sample(1) 
for col_name in names_df.columns:
    print(f"{col_name}: {test[col_name].first()}")

tax_id: 2231794
name_txt: Massjukichlorella epiphytica
unique_name: 
name_class: scientific name


now let's form the lineage strings


In [77]:
sci_name_df = names_df.filter(pl.col("name_class") == "scientific name").select(
    ["tax_id", "name_txt"]  ).rename({"name_txt": "scientific_name"}).unique()
# sci_name_df.null_count()

non tree accelrated

In [87]:
def build_all_lineages(nodes_df, sci_name_df):
    """Build lineage strings for all tax_ids efficiently using polars operations"""
    
    # Join nodes with scientific names to get name for each tax_id
    nodes_with_names = nodes_df.join(
        sci_name_df, 
        on="tax_id", 
        how="left"
    ).select(["tax_id", "parent_tax_id", "scientific_name"])
    
    # Build a dictionary for faster lookups
    tax_to_parent = dict(zip(
        nodes_with_names["tax_id"].to_list(),
        nodes_with_names["parent_tax_id"].to_list()
    ))
    tax_to_name = dict(zip(
        nodes_with_names["tax_id"].to_list(),
        nodes_with_names["scientific_name"].to_list()
    ))
    
    def get_lineage(tax_id):
        """Get lineage for a single tax_id"""
        if tax_id not in tax_to_name or tax_to_name[tax_id] is None:
            return ""
            
        lineage = []
        current = tax_id
        visited = set()
        
        while current in tax_to_parent and current not in visited and current != 1:
            visited.add(current)
            name = tax_to_name.get(current)
            if name:
                lineage.append(name)
            
            parent = tax_to_parent[current]
            if parent == current:  # Root case
                break
            current = parent
        
        return ";".join(reversed(lineage))
    
    # Get all unique tax_ids
    all_tax_ids = sci_name_df["tax_id"].unique().sort().to_list()
    print(f"Building lineages for {len(all_tax_ids)} tax_ids...")
    
    # Build lineages
    lineages = []
    for tax_id in tqdm(all_tax_ids):
        lineage = get_lineage(tax_id)
        lineages.append(lineage)
    
    # Create result DataFrame
    result_df = pl.DataFrame({
        "tax_id": all_tax_ids,
        "lineage": lineages
    })
    
    return result_df

# Build all lineages
lineage_df = build_all_lineages(nodes_df, sci_name_df)
print(f"Built {lineage_df.height} lineage strings")

# Show some examples with non-empty lineages
non_empty_lineages = lineage_df.filter(pl.col("lineage") != "")
print(f"\nFound {non_empty_lineages.height} non-empty lineages")
print("\nExample lineages:")
print(non_empty_lineages.head(10))

Building lineages for 2708826 tax_ids...


  0%|          | 0/2708826 [00:00<?, ?it/s]

Built 2708826 lineage strings

Found 2708825 non-empty lineages

Example lineages:
shape: (10, 2)
┌────────┬─────────────────────────────────┐
│ tax_id ┆ lineage                         │
│ ---    ┆ ---                             │
│ i64    ┆ str                             │
╞════════╪═════════════════════════════════╡
│ 2      ┆ cellular organisms;Bacteria     │
│ 6      ┆ cellular organisms;Bacteria;Ps… │
│ 7      ┆ cellular organisms;Bacteria;Ps… │
│ 9      ┆ cellular organisms;Bacteria;Ps… │
│ 10     ┆ cellular organisms;Bacteria;Ps… │
│ 11     ┆ cellular organisms;Bacteria;Ba… │
│ 13     ┆ cellular organisms;Bacteria;Ps… │
│ 14     ┆ cellular organisms;Bacteria;Ps… │
│ 16     ┆ cellular organisms;Bacteria;Ps… │
│ 17     ┆ cellular organisms;Bacteria;Ps… │
└────────┴─────────────────────────────────┘


In [79]:
# Create taxonomy tree
print("Creating TaxonomyTree with NCBI nodes...")

# Merge nodes with scientific names for the tree
nodes_with_names = nodes_df.join(
    sci_name_df, 
    on="tax_id", 
    how="left"
).select(["tax_id", "parent_tax_id", "rank", "scientific_name"])

# Create tree
tax_tree = TaxonomyTree(nodes_with_names)

Creating TaxonomyTree with NCBI nodes...


In [80]:
mini_genebank["total_gene_count"].null_count()

16885

In [81]:
# Set data availability based on your GenBank assembly data
# Filter to only include entries with actual genome/transcriptome data

data_available_df = mini_genebank.with_columns(
    pl.when(
        pl.col("total_gene_count") > 0).then(pl.lit(True).alias("has_genes")).otherwise(pl.lit(False).alias("has_genes")),
    pl.when(pl.col("protein_coding_gene_count") > 0).then(pl.lit(1).alias("has_proteins")).otherwise(pl.lit(0).alias("has_proteins")),
)
            
data_available_df

assembly_accession,bioproject,biosample,wgs_master,refseq_category,ncbi_taxonid,species_taxid,organism_name,infraspecific_name,isolate,version_status,assembly_level,release_type,genome_rep,seq_rel_date,asm_name,asm_submitter,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,asm_not_live_date,assembly_type,group,genome_size,genome_size_ungapped,gc_percent,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id,md5sums_path,has_genes,has_proteins
str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,f64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,bool,i32
"""GCA_038318695.1""",null,null,null,null,1949030,2955465,"""Influenza B virus (B/Illinois/…","""strain=B/Illinois/22/2016""",null,"""latest""","""Complete Genome""","""Major""","""Full""","""2017-02-06""","""ASM3831869v1""","""WHO Collaborating Center for S…",null,null,"""https://ftp.ncbi.nlm.nih.gov/g…",null,null,null,"""haploid""","""viral""",14398,14398,40.0,8,8,8,"""WHO Collaborating Center for S…","""Annotation submitted by WHO Co…","""2017-02-06""",11,11,0,null,"""https://ftp.ncbi.nlm.nih.gov/g…",true,1
"""GCA_041200105.1""","""PRJNA1134181""","""SAMN42395754""",null,null,3234140,3234140,"""Flavobacterium sp. WC2429""","""strain=WC2429""",null,"""latest""","""Chromosome""","""Major""","""Full""","""2024-08-11""","""ASM4120010v1""","""Wellesley College""","""GCF_041200105.1""","""identical""","""https://ftp.ncbi.nlm.nih.gov/g…",null,null,null,"""haploid""","""bacteria""",4043412,4043312,33.0,1,1,1,"""NCBI""","""NCBI Prokaryotic Genome Annota…","""2024-08-01""",3466,3375,78,null,"""https://ftp.ncbi.nlm.nih.gov/g…",true,1
"""GCA_007989705.1""","""PRJDB3962""","""SAMD00166011""","""BJUG00000000.1""",null,417368,417368,"""Enterococcus thailandicus""","""strain=NBRC 101867""",null,"""latest""","""Contig""","""Major""","""Full""","""2019-08-01""","""ASM798970v1""","""National Institute of Technolo…","""GCF_007989705.1""","""identical""","""https://ftp.ncbi.nlm.nih.gov/g…",null,"""assembly from type material""",null,"""haploid""","""bacteria""",2783774,2783774,36.5,0,36,36,"""National Institute of Technolo…","""Annotation submitted by Nation…","""2023-08-24""",2641,2597,44,null,"""https://ftp.ncbi.nlm.nih.gov/g…",true,1
"""GCA_001996065.1""","""PRJNA368798""","""SAMN06272993""","""MUFG00000000.1""",null,1909453,1909453,"""Salinivibrio sp. MA351""","""strain=MA351""",null,"""latest""","""Contig""","""Major""","""Full""","""2017-02-14""","""ASM199606v1""","""Universidad de Sevilla""","""GCF_001996065.1""","""identical""","""https://ftp.ncbi.nlm.nih.gov/g…",null,null,null,"""haploid""","""bacteria""",3308854,3308854,49.5,0,59,59,"""NCBI""","""NCBI Prokaryotic Genome Annota…","""2017-01-30""",3058,2931,93,"""28684561""","""https://ftp.ncbi.nlm.nih.gov/g…",true,1
"""GCA_014163945.1""","""PRJNA648432""","""SAMN15642170""","""JACJEG000000000.1""",null,2760948,2760948,"""Pseudoalteromonas sp. SR41-6""","""strain=SR41-6""",null,"""latest""","""Scaffold""","""Major""","""Full""","""2020-08-11""","""ASM1416394v1""","""University of Alaska Fairbanks""","""GCF_014163945.1""","""identical""","""https://ftp.ncbi.nlm.nih.gov/g…",null,null,null,"""haploid""","""bacteria""",5243740,5242440,40.5,0,284,297,"""NCBI""","""NCBI Prokaryotic Genome Annota…","""2020-08-06""",4775,4563,121,null,"""https://ftp.ncbi.nlm.nih.gov/g…",true,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""GCA_013107245.1""","""PRJNA596592""","""SAMN13631914""","""WVJT00000000.1""",null,2690675,2690675,"""Roseobacter sp. HKCCD9117-2""","""strain=HKCCD9117-2""",null,"""latest""","""Contig""","""Major""","""Full""","""2020-05-15""","""ASM1310724v1""","""Chinese University of Hong Kon…","""GCF_013107245.1""","""identical""","""https://ftp.ncbi.nlm.nih.gov/g…",null,null,null,"""haploid""","""b

In [82]:
data_available_df = data_available_df.filter(pl.col("has_genes")).select(["ncbi_taxonid","ftp_path"]).rename({"ncbi_taxonid": "tax_id"})
data_available_df


tax_id,ftp_path
i64,str
1949030,"""https://ftp.ncbi.nlm.nih.gov/g…"
3234140,"""https://ftp.ncbi.nlm.nih.gov/g…"
417368,"""https://ftp.ncbi.nlm.nih.gov/g…"
1909453,"""https://ftp.ncbi.nlm.nih.gov/g…"
2760948,"""https://ftp.ncbi.nlm.nih.gov/g…"
…,…
1964747,"""https://ftp.ncbi.nlm.nih.gov/g…"
3156266,"""https://ftp.ncbi.nlm.nih.gov/g…"
2690675,"""https://ftp.ncbi.nlm.nih.gov/g…"


In [265]:
print(f"Found {data_available_df.height} tax_ids with available assembly data")
tax_tree._set_data_availability(data_df=data_available_df)

print("TaxonomyTree ready for neighbor finding!")

Found 165650 tax_ids with available assembly data
Set data availability for 165650 tax_ids
TaxonomyTree ready for neighbor finding!


In [264]:
# Test the tree with some example queries
# Get some test tax_ids from your rRNA data that might not have assemblies
test_tax_ids = rrna_df["taxid"].cast(pl.Int64).unique().limit(50).to_list()
print(f"Testing with tax_ids: {test_tax_ids}")

for tax_id in test_tax_ids:
    print(f"\n--- Tax ID {tax_id} ---")
    try:
        result = tax_tree.find_nearest_with_data(tax_id, max_distance=5)
        
        if result['self_has_data']:
            print(f"✅ {tax_id} has data directly!")
        else:
            if result['ancestor']:
                ancestor = result['ancestor']
                print(f"🔼 Nearest ancestor with data: {ancestor['name']} (tax_id: {ancestor['tax_id']}, distance: {ancestor['distance']})")
            
            if result['descendants']:
                print(f"🔽 Descendants with data ({len(result['descendants'])}):")
                for desc in result['descendants'][:3]:  # Show first 3
                    print(f"   - {desc['name']} (tax_id: {desc['tax_id']}, distance: {desc['distance']})")
            
            if not result['ancestor'] and not result['descendants']:
                print("❌ No relatives with data found within distance 5")
                
    except Exception as e:
        print(f"❌ Error processing tax_id {tax_id}: {e}")

print("\nTree structure ready for production use!")

Testing with tax_ids: [0, 7, 9, 17, 21, 23, 24, 25, 33, 34, 35, 38, 41, 43, 45, 48, 51, 52, 54, 56, 61, 63, 65, 69, 83, 84, 86, 89, 94, 100, 102, 103, 104, 106, 108, 110, 114, 120, 122, 124, 128, 134, 139, 144, 148, 149, 150, 151, 152, 156]

--- Tax ID 0 ---
❌ No relatives with data found within distance 5

--- Tax ID 7 ---
🔽 Descendants with data (1):
   - Azorhizobium caulinodans ORS 571 (tax_id: 438753, distance: 1)

--- Tax ID 9 ---
✅ 9 has data directly!

--- Tax ID 17 ---
✅ 17 has data directly!

--- Tax ID 21 ---
❌ No relatives with data found within distance 5

--- Tax ID 23 ---
✅ 23 has data directly!

--- Tax ID 24 ---
✅ 24 has data directly!

--- Tax ID 25 ---
✅ 25 has data directly!

--- Tax ID 33 ---
✅ 33 has data directly!

--- Tax ID 34 ---
✅ 34 has data directly!

--- Tax ID 35 ---
✅ 35 has data directly!

--- Tax ID 38 ---
❌ No relatives with data found within distance 5

--- Tax ID 41 ---
✅ 41 has data directly!

--- Tax ID 43 ---
✅ 43 has data directly!

--- Tax ID 4

In [104]:
# Reload the updated TaxonomyTree module
import sys

# Remove cached module
if 'rolypoly.utils.bio.trees' in sys.modules:
    del sys.modules['rolypoly.utils.bio.trees']
    

# Reimport
from rolypoly.utils.bio.trees import TaxonomyTree

print("✅ TaxonomyTree module reloaded with enhanced features!")

✅ TaxonomyTree module reloaded with enhanced features!


In [90]:
# Recreate the tree with priority columns specified
print("Creating enhanced TaxonomyTree with priority columns...")

# Prepare data with full metadata
data_available_enhanced = mini_genebank.filter(
    pl.col("total_gene_count") > 0
).select([
    pl.col("ncbi_taxonid").alias("tax_id"),
    "ftp_path",
    "protein_coding_gene_count",
    "total_gene_count",
    "genome_size",
    "assembly_level",
    "organism_name"
])

print(f"Data available for {data_available_enhanced.height} tax_ids with metadata")

# Create tree with priority columns
tax_tree_enhanced = TaxonomyTree(
    nodes_with_names,
    data_availability_df=data_available_enhanced,
    priority_columns=['protein_coding_gene_count', 'genome_size']  # Prioritize by gene count, then genome size
)

print("\n✅ Enhanced tree created!")

Creating enhanced TaxonomyTree with priority columns...
Data available for 165650 tax_ids with metadata
Built tree with 2708826 nodes, 2457509 leaf nodes
Set data availability for 165650 tax_ids
Stored metadata for 165650 tax_ids

✅ Enhanced tree created!
Built tree with 2708826 nodes, 2457509 leaf nodes
Set data availability for 165650 tax_ids
Stored metadata for 165650 tax_ids

✅ Enhanced tree created!


In [91]:
# Feature 1: Get comprehensive tree statistics
print("="*70)
print("FEATURE 1: COMPREHENSIVE TREE STATISTICS")
print("="*70)

tax_tree_enhanced.print_stats()

FEATURE 1: COMPREHENSIVE TREE STATISTICS
TAXONOMY TREE STATISTICS

Tree Structure:
  Total nodes:          2,708,826
  Leaf nodes:           2,457,509
  Internal nodes:       251,317

Data Availability:
  Nodes with data:      165,650
  Leaves with data:     161,573
  Leaves without data:  2,295,936
  Internal nodes w/data:4,077

Coverage:
  All nodes:            6.12%
  Leaf nodes only:      6.57%

Cache:
  Cached queries:       0

Priority columns: protein_coding_gene_count, genome_size

Top ranks by count:
  species             : 2,223,790 total (83,775 with data,   3.8%)
  no rank             :  253,736 total (58,417 with data,  23.0%)
  genus               :  113,698 total (     0 with data,   0.0%)
  strain              :   46,928 total (22,400 with data,  47.7%)
  subspecies          :   30,788 total (   404 with data,   1.3%)
  family              :   10,817 total (     0 with data,   0.0%)
  varietas            :   10,355 total (    20 with data,   0.2%)
  subfamily           

In [92]:
# Feature 2: Batch query for multiple tax_ids with statistics
print("\n" + "="*70)
print("FEATURE 2: BATCH QUERIES WITH STATISTICS")
print("="*70)

# Get a sample of tax_ids from rrna_df
sample_tax_ids = rrna_df["taxid"].cast(pl.Int64).unique().sample(100).to_list()

print(f"\nQuerying {len(sample_tax_ids)} tax_ids from rRNA data...")

# Batch query with stats
results, stats = tax_tree_enhanced.find_nearest_with_data_batch(
    tax_ids=sample_tax_ids,
    max_distance=10,
    leaves_only=False,  # Include all nodes
    return_stats=True
)

print("\n📊 Batch Query Statistics:")
print(f"  Total queried:        {stats['total_queried']:,}")
print(f"  Self has data:        {stats['self_has_data']:,}")
print(f"  Ancestor found:       {stats['ancestor_found']:,}")
print(f"  Descendant found:     {stats['descendant_found']:,}")
print(f"  No match found:       {stats['no_match_found']:,}")

print("\n📏 Distance Distribution:")
for distance in sorted(stats['distance_distribution'].keys())[:10]:
    count = stats['distance_distribution'][distance]
    print(f"  Distance {distance}: {count:,} matches")

if stats['unmatched_tax_ids']:
    print(f"\n❌ Unmatched tax_ids (first 10): {stats['unmatched_tax_ids'][:10]}")


FEATURE 2: BATCH QUERIES WITH STATISTICS

Querying 100 tax_ids from rRNA data...

📊 Batch Query Statistics:
  Total queried:        100
  Self has data:        19
  Ancestor found:       0
  Descendant found:     2
  No match found:       79

📏 Distance Distribution:
  Distance 0: 19 matches
  Distance 1: 2 matches

❌ Unmatched tax_ids (first 10): [413816, 244960, 870734, 304739, 1973154, 1769086, 979731, 573973, 639593, 297705]


In [263]:
# Feature 3: Priority-based neighbor selection
print("\n" + "="*70)
print("FEATURE 3: PRIORITY-BASED NEIGHBOR SELECTION")
print("="*70)

# Find a tax_id without data to demonstrate priority selection
test_tax_id = None
for tid in sample_tax_ids[:50]:
    if tid not in tax_tree_enhanced.data_available and tid in tax_tree_enhanced.parent_map:
        test_tax_id = tid
        break

if test_tax_id:
    print(f"\nTesting with tax_id: {test_tax_id}")
    print(f"Name: {tax_tree_enhanced.name_map.get(test_tax_id, 'Unknown')}")
    
    # Find neighbors with default priority
    result_default = tax_tree_enhanced.find_nearest_with_data(test_tax_id, max_distance=5)
    
    print("\n🔍 Neighbors with DEFAULT priority (protein_coding_gene_count, genome_size):")
    if result_default['ancestor']:
        anc = result_default['ancestor']
        meta = anc.get('metadata', {})
        print(f"\n  Ancestor: {anc['name']} (distance: {anc['distance']})")
        print(f"    Protein genes: {meta.get('protein_coding_gene_count', 'N/A'):,}")
        print(f"    Genome size:   {meta.get('genome_size', 'N/A'):,}")
    
    if result_default['descendants']:
        print(f"\n  Descendants ({len(result_default['descendants'])} found):")
        for desc in result_default['descendants'][:3]:
            meta = desc.get('metadata', {})
            print(f"    {desc['name']} (distance: {desc['distance']})")
            print(f"      Protein genes: {meta.get('protein_coding_gene_count', 'N/A'):,}")
            print(f"      Genome size:   {meta.get('genome_size', 'N/A'):,}")
    
    # Override priority to prioritize genome_size over gene count
    result_override = tax_tree_enhanced.find_nearest_with_data(
        test_tax_id, 
        max_distance=5, 
        priority_override=['genome_size', 'protein_coding_gene_count']
    )
    
    print("\n🔍 Neighbors with OVERRIDE priority (genome_size, protein_coding_gene_count):")
    if result_override['ancestor']:
        anc = result_override['ancestor']
        meta = anc.get('metadata', {})
        print(f"\n  Ancestor: {anc['name']} (distance: {anc['distance']})")
        print(f"    Genome size:   {meta.get('genome_size', 'N/A'):,}")
        print(f"    Protein genes: {meta.get('protein_coding_gene_count', 'N/A'):,}")
else:
    print("Could not find a suitable test tax_id without data")


FEATURE 3: PRIORITY-BASED NEIGHBOR SELECTION

Testing with tax_id: 413816
Name: Halorubrum halophilum

🔍 Neighbors with DEFAULT priority (protein_coding_gene_count, genome_size):

🔍 Neighbors with OVERRIDE priority (genome_size, protein_coding_gene_count):


In [262]:
# Feature 4: Leaf-only processing (optimized for rRNA mapping use case)
print("\n" + "="*70)
print("FEATURE 4: LEAF-ONLY PROCESSING")
print("="*70)

# Since rRNA sequences can only come from extant organisms (leaves),
# we can optimize by only processing leaf nodes

print("\nProcessing only leaf nodes from rRNA data...")

# Get all unique tax_ids from rRNA data
all_rrna_tax_ids = rrna_df["taxid"].cast(pl.Int64).unique().to_list()
print(f"Total rRNA tax_ids: {len(all_rrna_tax_ids):,}")

# Check how many are leaves
leaves_in_rrna = [tid for tid in all_rrna_tax_ids if tid in tax_tree_enhanced.leaf_nodes]
print(f"Leaf nodes in rRNA: {len(leaves_in_rrna):,} ({len(leaves_in_rrna)/len(all_rrna_tax_ids)*100:.1f}%)")

# Batch query leaves only with statistics
print("\nQuerying sample of leaf nodes...")
sample_leaves = leaves_in_rrna[:200] if len(leaves_in_rrna) > 200 else leaves_in_rrna

results_leaves, stats_leaves = tax_tree_enhanced.find_nearest_with_data_batch(
    tax_ids=sample_leaves,
    max_distance=10,
    leaves_only=True,  # Only process leaves
    return_stats=True
)

print("\n📊 Leaf-Only Query Statistics:")
print(f"  Total queried:        {stats_leaves['total_queried']:,}")
print(f"  Self has data:        {stats_leaves['self_has_data']:,}")
print(f"  Ancestor found:       {stats_leaves['ancestor_found']:,}")
print(f"  Descendant found:     {stats_leaves['descendant_found']:,}")
print(f"  No match found:       {stats_leaves['no_match_found']:,}")

print("\n📏 Distance Distribution for Leaves:")
for distance in sorted(stats_leaves['distance_distribution'].keys())[:10]:
    count = stats_leaves['distance_distribution'][distance]
    print(f"  Distance {distance}: {count:,} matches")

# Calculate coverage for rRNA data
coverage_pct = ((stats_leaves['self_has_data'] + stats_leaves['ancestor_found']) / 
                stats_leaves['total_queried'] * 100)
print(f"\n✅ Coverage: {coverage_pct:.1f}% of rRNA leaf nodes have data or found ancestors with data")


FEATURE 4: LEAF-ONLY PROCESSING

Processing only leaf nodes from rRNA data...
Total rRNA tax_ids: 105,793
Leaf nodes in rRNA: 96,898 (91.6%)

Querying sample of leaf nodes...

📊 Leaf-Only Query Statistics:
  Total queried:        200
  Self has data:        152
  Ancestor found:       1
  Descendant found:     0
  No match found:       47

📏 Distance Distribution for Leaves:
  Distance 0: 152 matches
  Distance 1: 1 matches

✅ Coverage: 76.5% of rRNA leaf nodes have data or found ancestors with data


In [98]:
# Practical example: Build lookup table for rRNA sequences -> genomes
print("\n" + "="*70)
print("PRACTICAL APPLICATION: rRNA -> GENOME MAPPING TABLE")
print("="*70)

# Process ALL rRNA tax_ids (this might take a moment)
print("\nProcessing all rRNA tax_ids to build mapping table...")

all_results, all_stats = tax_tree_enhanced.find_nearest_with_data_batch(
    tax_ids=all_rrna_tax_ids,
    max_distance=4,
    leaves_only=False,  # Process all to see full picture
    return_stats=True
)



PRACTICAL APPLICATION: rRNA -> GENOME MAPPING TABLE

Processing all rRNA tax_ids to build mapping table...


In [261]:
# Build a mapping table
mapping_records = []
for tax_id, result in all_results.items():
    # Get rRNA info
    rrna_info = rrna_df.filter(pl.col("taxid").cast(pl.Int64) == tax_id).row(0, named=True) if tax_id in rrna_df["taxid"].cast(pl.Int64) else {}
    
    if result['self_has_data']:
        # Direct mapping
        mapping_records.append({
            'rrna_tax_id': tax_id,
            'rrna_organism': tax_tree_enhanced.name_map.get(tax_id, 'Unknown'),
            'genome_tax_id': tax_id,
            'genome_organism': tax_tree_enhanced.name_map.get(tax_id, 'Unknown'),
            'relationship': 'self',
            'distance': 0,
            'ftp_path': result['ancestor'].get('metadata', {}).get('ftp_path', ''),
            'protein_coding_genes': result['ancestor'].get('metadata', {}).get('protein_coding_gene_count', 0),
            'genome_size': result['ancestor'].get('metadata', {}).get('genome_size', 0)
        })
    elif result['ancestor']:
        # Map to ancestor
        anc = result['ancestor']
        mapping_records.append({
            'rrna_tax_id': tax_id,
            'rrna_organism': tax_tree_enhanced.name_map.get(tax_id, 'Unknown'),
            'genome_tax_id': anc['tax_id'],
            'genome_organism': anc['name'],
            'relationship': 'ancestor',
            'distance': anc['distance'],
            'ftp_path': anc.get('metadata', {}).get('ftp_path', ''),
            'protein_coding_genes': anc.get('metadata', {}).get('protein_coding_gene_count', 0),
            'genome_size': anc.get('metadata', {}).get('genome_size', 0)
        })
    elif result['descendants']:
        # Map to best descendant
        desc = result['descendants'][0]  # Already prioritized
        mapping_records.append({
            'rrna_tax_id': tax_id,
            'rrna_organism': tax_tree_enhanced.name_map.get(tax_id, 'Unknown'),
            'genome_tax_id': desc['tax_id'],
            'genome_organism': desc['name'],
            'relationship': 'descendant',
            'distance': desc['distance'],
            'ftp_path': desc.get('metadata', {}).get('ftp_path', ''),
            'protein_coding_genes': desc.get('metadata', {}).get('protein_coding_gene_count', 0),
            'genome_size': desc.get('metadata', {}).get('genome_size', 0)
        })


KeyboardInterrupt: 

## Alternative: Using bigtree library

Exploring the `bigtree` library which provides native Polars support for tree construction and manipulation.

In [118]:
# Try using bigtree library with polars_to_tree
# First, check if bigtree is installed, if not, install it
try:
    import bigtree  
    print(f"✅ bigtree version {bigtree.__version__} is installed")
except ImportError:
    print("📦 Installing bigtree...")
    import subprocess
    subprocess.run(["pip", "install", "bigtree"], check=True)
    import bigtree
    print(f"✅ bigtree version {bigtree.__version__} installed successfully")


✅ bigtree version 0.30.1 is installed


In [225]:
# Prepare data for bigtree
# bigtree expects a path column that shows the full hierarchical path
# We'll use our nodes_with_names DataFrame

print("Preparing data for bigtree's polars_to_tree...")
print(f"Starting with {nodes_with_names.height:,} nodes")

# Let's use nodes related to our rRNA data
tax_ids_bigtree = rrna_df["taxid"].cast(pl.Int64).unique().to_list()

# Get all ancestors for these nodes to build complete paths
def get_all_ancestors(tax_id, parent_map, max_depth=50):
    """Get all ancestors up to root"""
    ancestors = []
    current = tax_id
    depth = 0
    visited = set()
    
    while current in parent_map and depth < max_depth and current not in visited:
        visited.add(current)
        ancestors.append(current)
        parent = parent_map[current]
        if parent == current:  # Root
            break
        current = parent
        depth += 1
    
    return ancestors

# Build parent map for quick lookups
parent_map_bigtree = dict(zip(nodes_with_names["tax_id"], nodes_with_names["parent_tax_id"]))

# Get all nodes needed 
all_needed_ids = set()
for tid in tax_ids_bigtree:
    if tid in parent_map_bigtree:
        ancestors = get_all_ancestors(tid, parent_map_bigtree)
        all_needed_ids.update(ancestors)

print(f"Collected {len(all_needed_ids):,} nodes (including ancestors)")

# Filter to only needed nodes
bigtree_df = nodes_with_names.filter(pl.col("tax_id").is_in(all_needed_ids))
print(f"Filtered DataFrame has {bigtree_df.height:,} rows")

Preparing data for bigtree's polars_to_tree...
Starting with 2,708,826 nodes
Collected 148,236 nodes (including ancestors)
Filtered DataFrame has 148,236 rows
Collected 148,236 nodes (including ancestors)
Filtered DataFrame has 148,236 rows


In [226]:
bigtree_df

tax_id,parent_tax_id,rank,scientific_name
i64,i64,str,str
1,1,"""no rank""","""root"""
2,131567,"""domain""","""Bacteria"""
6,335928,"""genus""","""Azorhizobium"""
7,6,"""species""","""Azorhizobium caulinodans"""
9,32199,"""species""","""Buchnera aphidicola"""
…,…,…,…
3467296,1760,"""order""","""Allobogoriellales"""
3471053,2841720,"""genus""","""Tibetiodes"""
3472296,159987,"""family""","""Neocamarosporiaceae"""


bigtree expects the root node parent to be null, not itself

In [ ]:
bigtree_df = bigtree_df.with_columns(
    pl.when(pl.col("tax_id") == 1)
      .then(pl.lit(None))
      .otherwise(pl.col("parent_tax_id"))
      .alias("parent_tax_id")
) 

In [ ]:
import bigtree as bt
test_tree = bt.polars_to_tree_by_relation(
    allow_duplicates=True,
    child_col="tax_id",
    parent_col="parent_tax_id",
    data=bigtree_df.select(["tax_id","parent_tax_id"]),
    )

In [244]:
test_tree

Node(/1, )

In [ ]:
# # Alternative: Using NetworkX for more advanced tree operations
# # NetworkX is excellent for complex graph algorithms and visualization

# import networkx as nx
# nx.from_nested_tuple
# def create_networkx_taxonomy_tree(nodes_df, data_availability_set):
#     """Create NetworkX directed graph for taxonomy tree"""
#     # G = nx.DiGraph(incoming_graph_data=)
    
#     # Add nodes with attributes
#     for row in nodes_df.iter_rows(named=True):
#         tax_id = row['tax_id']
#         G.add_node(tax_id, 
#                     parent=row['parent_tax_id'],
#                     rank=row.get('rank', ''),
#                     name=row.get('scientific_name', ''),
#                     has_data=tax_id in data_availability_set)
        
#         # Add edges (parent -> child relationships)
#         parent_id = row['parent_tax_id']
#         if parent_id != tax_id:  # Avoid self-loops
#             G.add_edge(parent_id, tax_id)
    
#     return G

# def find_nearest_data_nx(G, tax_id, max_distance=10):
#     """Find nearest neighbors with data using NetworkX algorithms"""
#     if not G.has_node(tax_id):
#         return None
        
#     if G.nodes[tax_id]['has_data']:
#         return {'self': tax_id, 'distance': 0}
    
#     # Find ancestors (predecessors in directed path to root)
#     ancestors = []
#     current = tax_id
#     distance = 0
#     while distance < max_distance:
#         predecessors = list(G.predecessors(current))
#         if not predecessors:
#             break
#         parent = predecessors[0]  # Should be only one parent
#         distance += 1
#         if G.nodes[parent]['has_data']:
#             ancestors.append({'tax_id': parent, 'distance': distance, 
#                             'name': G.nodes[parent]['name']})
#             break
#         current = parent
    
#     # Find descendants using BFS
#     descendants = []
#     for node in nx.bfs_tree(G, tax_id, max_depth=max_distance):
#         if node != tax_id and G.nodes[node]['has_data']:
#             distance = nx.shortest_path_length(G, tax_id, node)
#             descendants.append({'tax_id': node, 'distance': distance,
#                                 'name': G.nodes[node]['name']})
    
#     return {
#         'ancestors': ancestors,
#         'descendants': sorted(descendants, key=lambda x: x['distance'])
#     }

# test = create_networkx_taxonomy_tree(nodes_df,data_available_df)

## Alternative: Using tskit (Tree Sequence Kit)

`tskit` is a high-performance library originally designed for population genetics, but works great for large taxonomy trees. It uses efficient C implementations and can handle millions of nodes.

In [245]:
# Import tskit and prepare taxonomy data for tskit format
import tskit
import numpy as np

print("Preparing taxonomy tree for tskit...")

# Use the same subset as bigtree for comparison
tskit_df = bigtree_df.select(["tax_id", "parent_tax_id", "scientific_name", "rank"])

# Create ID mapping: tskit requires integer node IDs starting from 0
all_tax_ids = pl.concat([
    tskit_df["tax_id"], 
    tskit_df["parent_tax_id"]
]).unique().sort()

# Create mapping from tax_id to sequential integer
id_map = all_tax_ids.to_frame("tax_id").with_row_index("node_id")
print(f"Created mapping for {id_map.height:,} unique nodes")

# Join to get integer IDs - do in two separate steps to avoid column conflicts
# First join for child IDs
tskit_edges = tskit_df.join(id_map, left_on="tax_id", right_on="tax_id", how="left")
tskit_edges = tskit_edges.rename({"node_id": "child"})

# Second join for parent IDs
parent_map = id_map.rename({"node_id": "parent", "tax_id": "parent_tax_id"})
tskit_edges = tskit_edges.join(parent_map, on="parent_tax_id", how="left")

# Select final columns
tskit_edges = tskit_edges.select(["child", "parent", "scientific_name", "rank"])

print(f"Prepared {tskit_edges.height:,} edges for tskit")
print(f"Sample edges:\n{tskit_edges.head(3)}")

Preparing taxonomy tree for tskit...
Created mapping for 148,237 unique nodes
Prepared 148,236 edges for tskit
Sample edges:
shape: (3, 4)
┌───────┬────────┬─────────────────┬─────────┐
│ child ┆ parent ┆ scientific_name ┆ rank    │
│ ---   ┆ ---    ┆ ---             ┆ ---     │
│ u32   ┆ u32    ┆ str             ┆ str     │
╞═══════╪════════╪═════════════════╪═════════╡
│ 1     ┆ null   ┆ root            ┆ no rank │
│ 2     ┆ 28572  ┆ Bacteria        ┆ domain  │
│ 3     ┆ 55375  ┆ Azorhizobium    ┆ genus   │
└───────┴────────┴─────────────────┴─────────┘


In [247]:
# Build tskit TableCollection and TreeSequence (with explicit root handling)
print("Building tskit tree structure...")

# Initialize TableCollection
tables = tskit.TableCollection(sequence_length=1.0)

print("Computing node depths...")

# Build parent-child map, excluding self-loops
parent_child_map = defaultdict(list)
all_nodes = set(range(id_map.height))
children_set = set()

for row in tskit_edges.filter(pl.col("parent").is_not_null()).iter_rows(named=True):
    parent = row['parent']
    child = row['child']
    if parent != child:  # Exclude self-loops (this is the root)
        parent_child_map[parent].append(child)
        children_set.add(child)

# Find roots: nodes that are never children OR nodes where parent == child
roots = []
for row in tskit_edges.iter_rows(named=True):
    child = row['child']
    parent = row['parent']
    # Root if: no parent, self-parent, or never appears as a child
    if parent is None or parent == child or child not in children_set:
        roots.append(child)

roots = list(set(roots))  # Deduplicate
print(f"Found {len(roots)} root node(s): {roots}")

if not roots:
    # Fallback: just use node 0 as root
    print("Warning: No clear root found, using node 0")
    roots = [0]

# BFS to assign depths - start from roots at depth 0
depth_map = {}
queue = deque([(root, 0) for root in roots])
visited = set()

while queue:
    node, depth = queue.popleft()
    if node in visited:
        continue
    visited.add(node)
    depth_map[node] = depth
    
    # Add children at next depth
    for child in parent_child_map.get(node, []):
        if child not in visited:
            queue.append((child, depth + 1))

# Assign depth to any unvisited nodes (orphans)
max_depth = max(depth_map.values()) if depth_map else 0
for node_id in all_nodes:
    if node_id not in depth_map:
        depth_map[node_id] = max_depth + 1

# Create time array: time = max_depth - depth (so root has highest time)
max_depth = max(depth_map.values())
times = np.array([max_depth - depth_map.get(i, 0) for i in range(id_map.height)], dtype=np.float64)

print(f"Depth range: 0 to {max_depth}")
print(f"Time range: {times.min():.1f} to {times.max():.1f}")
print(f"Nodes at depth 0 (roots): {sum(1 for d in depth_map.values() if d == 0)}")

# Add nodes
tables.nodes.set_columns(
    flags=np.zeros(id_map.height, dtype=np.uint32),
    time=times

)
# tables.nodes.set_columns(
#     time=np.zeros(id_map.height), # Placeholder time
#     flags=np.zeros(id_map.height, dtype=np.uint32)
# )

print(f"Added {id_map.height:,} nodes")

# Add edges (exclude self-loops)
edges_df = tskit_edges.filter(
    pl.col("parent").is_not_null() & 
    (pl.col("parent") != pl.col("child"))
)

print(f"Edges before filtering: {tskit_edges.height:,}")
print(f"Edges after filtering: {edges_df.height:,}")

if edges_df.height > 0:
    tables.edges.set_columns(
        left=np.zeros(edges_df.height, dtype=np.float64),
        right=np.ones(edges_df.height, dtype=np.float64),
        parent=edges_df["parent"].cast(pl.Int32).to_numpy(),
        child=edges_df["child"].cast(pl.Int32).to_numpy()
    )
    print(f"Added {edges_df.height:,} edges to tree")

# Sort and build index
print("Sorting tables...")
tables.sort()
print("Building index...")
tables.build_index()

# Create tree sequence
ts = tables.tree_sequence()

print("\n✅ Created tskit TreeSequence:")
print(f"   Nodes: {ts.num_nodes:,}")
print(f"   Edges: {ts.num_edges:,}")
print(f"   Trees: {ts.num_trees}")

Building tskit tree structure...
Computing node depths...
Found 1 root node(s): [1]
Depth range: 0 to 36
Time range: 0.0 to 36.0
Nodes at depth 0 (roots): 1
Added 148,237 nodes
Edges before filtering: 148,236
Edges after filtering: 148,235
Added 148,235 edges to tree
Sorting tables...
Building index...

✅ Created tskit TreeSequence:
   Nodes: 148,237
   Edges: 148,235
   Trees: 1
Depth range: 0 to 36
Time range: 0.0 to 36.0
Nodes at depth 0 (roots): 1
Added 148,237 nodes
Edges before filtering: 148,236
Edges after filtering: 148,235
Added 148,235 edges to tree
Sorting tables...
Building index...

✅ Created tskit TreeSequence:
   Nodes: 148,237
   Edges: 148,235
   Trees: 1


In [ ]:
# Explore tskit tree operations
print("="*70)
print("TSKIT TREE OPERATIONS")
print("="*70)

# Get the first (and only) tree
tree = ts.first()

print("\n1️⃣ Tree properties:")
print(f"   Root: {tree.root}")
print(f"   Num roots: {tree.num_roots}")
print(f"   Total branch length: {tree.total_branch_length:.2f}")

# Find MRCA (Most Recent Common Ancestor) - useful for taxonomy!
sample_nodes = [5, 10, 15]  # Example node IDs
if all(n < ts.num_nodes for n in sample_nodes):
    mrca = tree.mrca(*sample_nodes)
    print(f"\n2️⃣ MRCA of nodes {sample_nodes}: {mrca}")

# Get parent/child relationships
print("\n3️⃣ Tree traversal examples:")
# Get children of root
root_children = tree.get_children(1)
print(f"   Root node {tree.root} has {len(root_children)} children: {root_children[:5]}")


TSKIT TREE OPERATIONS

1️⃣ Tree properties:
   Root: -1
   Num roots: 0
   Total branch length: 0.00

2️⃣ MRCA of nodes [5, 10, 15]: 757

3️⃣ Tree traversal examples:
   Root node -1 has 1 children: (28572,)


In [ ]:
# Get parent of a node
sample_node = 10
if sample_node < ts.num_nodes:
    parent = tree.parent(sample_node)
    print(f"   Node {sample_node} has parent: {parent}")

# Get all samples under a node
print("\n4️⃣ Samples (leaves) under root:")
samples_under_root = list(tree.samples(tree.root))
print(f"   Found {len(samples_under_root)} leaf nodes")
print(f"   First 10: {samples_under_root[:10]}")

# Path from node to root (lineage!)
sample_node = 20
if sample_node < ts.num_nodes:
    print(f"\n5️⃣ Path from node {sample_node} to root (lineage):")
    current = sample_node
    path = []
    while current != tskit.NULL:
        path.append(current)
        current = tree.parent(current)
    print(f"   Path ({len(path)} nodes): {path[:10]}{'...' if len(path) > 10 else ''}")

print("\n✅ tskit provides fast tree traversal operations!")

   Node 10 has parent: 6457

4️⃣ Samples (leaves) under root:
   Found 0 leaf nodes
   First 10: []

5️⃣ Path from node 20 to root (lineage):
   Path (7 nodes): [20, 6461, 139261, 147261, 2, 28572, 1]

✅ tskit provides fast tree traversal operations!
